In [90]:
from dataclasses import dataclass

import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from scipy import optimize

In [133]:
@dataclass(frozen=True)
class AdditionalLine:
    x: np.ndarray
    a: float
    b: float
    name_method: str
    color: str = "green"
    width: float = 7
    type_: str = 'liner'

    def generate_liner(self) -> tuple[np.ndarray]:
        return self.a * self.x + self.b

    def generate_rational(self) -> tuple[np.ndarray]:
        return self.a / (1 + self.b * self.x)

    def create_figure(self) -> go.Scatter:
        return go.Scatter(
            x=self.x,
            y=self.generate_liner() if self.type_ == 'liner' else self.generate_rational(),
            name=self.name_method,
            line=go.scatter.Line(color=self.color, width=self.width)
        )


def generate_x(size: int = 100) -> np.ndarray:
    k = np.linspace(0, size, size)
    return k / size


def generate_noise_data(size: int = 100) -> tuple[np.ndarray, np.ndarray]:
    alpha, beta, delta = (np.random.sample(size) for _ in range(3))
    x = generate_x(size)
    return x, alpha * x + beta + delta


def liner_approx(x: np.ndarray, a: float, b: float) -> tuple[np.ndarray]:
    return a * x + b


def visualize(
        x: np.ndarray,
        y: np.ndarray,
        title: str | None = None,
        additional_line: AdditionalLine | None = None,
) -> go.Figure:
    coef = optimize.curve_fit(liner_approx, x, y)
    a, b = coef[0]
    y_lsm = liner_approx(x, a, b)

    fig = px.scatter(
        x=x, y=y,
        template="plotly_dark",
        width=1500, height=1000,
    )
    fig.update_layout(
        title="🥹" if title is None else title,
        xaxis_title="X",
        yaxis_title="Y",
        font=dict(
            family="Courier New, monospace",
            size=18,
        )
    )
    fig.update_traces(
        marker=dict(
            size=12,
            line=dict(
                width=2,
                color='DarkSlateGrey'
            )
        ),
        selector=dict(mode='markers')
    )
    fig.add_trace(
        go.Scatter(
            x=x, y=y_lsm,
            name="Generative Straight",
            line=go.scatter.Line(color="gray", width=7),

        )
    )
    if additional_line is not None:
        fig.add_trace(
            additional_line.create_figure()
        )

    return fig


In [86]:
x, y = generate_noise_data()

# **_Optimization by Levenberg-Marquart method_**

## $F(x, a, b) = ax + b$

In [113]:
def liner_optimize_function(z, support_x):
    a_, b_ = z
    return a_ * support_x + b_


def function_being_optimized_lmm(z, support_x, support_y):
    return (liner_optimize_function(z, support_x) - support_y) ** 2

zero_condition = 1, 0
result_lm = optimize.least_squares(function_being_optimized_lmm, zero_condition, method='lm', args=(x, y), xtol=10 ** -3)
a_lm, b_lm = result_lm.x

additional_line_liner_lev_mar =  AdditionalLine(x=x, a=a_lm, b=b_lm, name_method="Levenberg-Marquart", width=5, color='#64FFD6')

figure = visualize(x=x, y=y, title="Optimization by Levenberg-Marquart method", additional_line=additional_line_liner_lev_mar)
figure.show()

# _Optimization by Newton_
## $F(x, a, b) = ax + b$

In [139]:
def function_being_optimized_disc(z, support_x, support_y):

    return np.sum((liner_optimize_function(z, support_x) - support_y) ** 2)

zero_condition_newton = 10 ** -3, 0.2
res_nm = optimize.newton(function_being_optimized_disc, zero_condition_newton, tol=10 ** -3, maxiter=10008, args=(x, y))
a_nwt, b_nwt = res_nm

additional_line_liner_newton =  AdditionalLine(x=x, a=a_nwt, b=b_nwt, name_method="Newton Method", width=5, color='#CFB0E4')

figure = visualize(x=x, y=y, title="Optimization by Newton", additional_line=additional_line_liner_newton)
figure.show()

# CG
## $F(x, a, b) = ax + b$

In [153]:
def fprime(x_, support_x, support_y):
    return optimize.approx_fprime(x_, function_being_optimized_disc, 10 ** -3, support_x, support_y)

zero_condition = 10 ** -3, 0
res_cg = optimize.minimize(function_being_optimized_disc, zero_condition, method='CG', tol=10 ** -3, jac=fprime, args=(x, y))
a, b = res_cg.x

additional_line_liner_cg =  AdditionalLine(x=x, a=a, b=b, name_method="Conjugate Gradient", width=4, color='#FF5656')
visualize(x=x, y=y, title="Minimization by Conjugate Gradient method", additional_line=additional_line_liner_cg).show()

# _Optimization by Levenberg-Marquart method_
## $F(x, a, b) = $ $\frac{a}{1 + bx}$

In [137]:
def rational_optimize_function(z, support_x):
    a_, b_ = z
    return a_ / (1 + support_x * b_)


def function_being_optimized_lmm_rational(z, support_x, support_y):
    return (rational_optimize_function(z, support_x) - support_y) ** 2

zero_condition = 1, 0
res_lm = optimize.least_squares(function_being_optimized_lmm_rational, zero_condition, method='lm', xtol=10 ** -3, args=(x, y))
a_lmr, b_lmr = res_lm.x

additional_line =  AdditionalLine(x=x, a=a_lmr, b=b_lmr, name_method="Levenberg-Marquart method", width=4, color='#FF6FC5', type_='rational')

visualize(x=x, y=y, title="Optimization by Levenberg-Marquart method", additional_line=additional_line).show()
